In [5]:
import Ipynb_importer
from a_CSPdarknet53 import DarknetConv2D, darknet_body,compose 

In [4]:
from functools import wraps

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import (Add, BatchNormalization, Concatenate,
                                     Conv2D, LeakyReLU, MaxPooling2D,
                                     UpSampling2D, ZeroPadding2D)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

### 定义CBL 基本模块

In [17]:
def DarknetConv2D_BN_LeakyReLU(*args, **kwargs):
    """Darknet Convolution2D followed by BatchNormalization and LeakyReLU."""
    
    no_bias_kwargs = {'use_bias': False}  # 没懂为啥用 no_bias
    no_bias_kwargs.update(kwargs)
    return compose(
        DarknetConv2D(*args, **no_bias_kwargs),
        BatchNormalization(),
        LeakyReLU(alpha=0.1)
    )

### 定义 yolo 主体
<img src="https://pic3.zhimg.com/80/v2-5251e9c0784871a37c693d53f7d57f92_1440w.jpg" alt="img" style="zoom:50%;" />

In [ ]:
def yolo_body(inputs, num_anchors, num_classes):
    # 1、通过CSPdarknet_body 获得三个有效特征层
    feat1, feat2, feat3 = darknet_body(inputs)
    
    # 2、获得 p5、p4、p3
    P5 = DarknetConv2D_BN_Leaky(512, (1, 1))(feat3)
    P4 = DarknetConv2D_BN_Leaky(256, (1,1))(feat2)
    P3 = DarknetConv2D_BN_Leaky(128, (1,1))(feat1)
    
    # 3.1、p5 neck部分处理
    P5 = DarknetConv2D_BN_Leaky(1024, (3, 3))(P5)
    P5 = DarknetConv2D_BN_Leaky(512, (1, 1))(P5)
    # 使用了SPP结构，即不同尺度的最大池化后堆叠。
    maxpool1 = MaxPooling2D(pool_size=(13, 13), strides=(1, 1), padding='same')(P5)
    maxpool2 = MaxPooling2D(pool_size=(9, 9), strides=(1, 1), padding='same')(P5)
    maxpool3 = MaxPooling2D(pool_size=(5, 5), strides=(1, 1), padding='same')(P5)
    P5 = Concatenate()([maxpool1, maxpool2, maxpool3, P5])
    P5 = DarknetConv2D_BN_Leaky(512, (1, 1))(P5)
    P5 = DarknetConv2D_BN_Leaky(1024, (3, 3))(P5)
    P5 = DarknetConv2D_BN_Leaky(512, (1, 1))(P5)
    
    # 3.2、p5 上采样与p4 实现 pan 结构
    P5_upsample = compose(DarknetConv2D_BN_Leaky(256, (1,1)), UpSampling2D(2))(P5)
    P4 = Concatenate()([P4, P5_upsample])
    
    # 4.1、p4 neck 部分处理
    P4 = make_five_convs(P4,256)
    
    # 4.2、p4 上采样与p3 实现 pan 结构
    P4_upsample = compose(DarknetConv2D_BN_Leaky(128, (1,1)), UpSampling2D(2))(P4)
    P3 = Concatenate()([P3, P4_upsample])
    
    # 5、p3 输出
    P3 = make_five_convs(P3,128)
    P3_output = DarknetConv2D_BN_Leaky(256, (3,3))(P3)
    P3_output = DarknetConv2D(num_anchors*(num_classes+5), (1,1), kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01))(P3_output)

    # 6、p4 输出
    P3_downsample = ZeroPadding2D(((1,0),(1,0)))(P3)
    P3_downsample = DarknetConv2D_BN_Leaky(256, (3,3), strides=(2,2))(P3_downsample)
    P4 = Concatenate()([P3_downsample, P4])
    P4 = make_five_convs(P4,256)
    P4_output = DarknetConv2D_BN_Leaky(512, (3,3))(P4)
    P4_output = DarknetConv2D(num_anchors*(num_classes+5), (1,1), kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01))(P4_output)
    
    # 7、p5输出
    P4_downsample = ZeroPadding2D(((1,0),(1,0)))(P4)
    P4_downsample = DarknetConv2D_BN_Leaky(512, (3,3), strides=(2,2))(P4_downsample)
    P5 = Concatenate()([P4_downsample, P5])
    P5 = make_five_convs(P5,512)
    P5_output = DarknetConv2D_BN_Leaky(1024, (3,3))(P5)
    P5_output = DarknetConv2D(num_anchors*(num_classes+5), (1,1), kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01))(P5_output)

    return Model(inputs, [P5_output, P4_output, P3_output])